# Collecting data for reactions procedures

````{warning} 
This Notebook can be run with any computer, no special requirements are needed since the completions are done through API calls.
````

````{admonition} Motivation 
:class: note

In this Notebook we aim to illustrate a simple demo on how to extract the different species involved in a organic reaction. The main aim is to develop a method to check if the data extracted by the model has the correct number of atoms in both sides of the reaction. To show the process, we have extracted three reactions from the *USPTO-ORD-100K* dataset. {cite}`Ai_2024`

To this end, we developed a simple `pydantic` class that we use to constrain the output of the model by using the package `instructor`. Then by the use of the tools of the `rdkit` package we will count the atoms from the SMILES and check if the condition is fullfilled.
````

## First steps

We begin by importing all the packages needed

In [2]:
import llmstructdata  # noqa: F401

import json
import time
import requests

from collections import defaultdict

from pydantic import BaseModel, Field
from typing import Optional, List
from litellm import OpenAI
import instructor

from rdkit import Chem
from rdkit.Chem.rdmolops import GetFormalCharge
import pubchempy as pcp
import periodictable

Now we load the data with the reaction's procedure. 

```` {margin}
The only requirement followed when choosing the reactions here studied was that the reactions must contain the name of the product. This is because some reactions from the USPTO-ORD-100K dataset describe the main product of the reaction as "the title compound", which is not ideal to our case.
````

In [ ]:
# Replace 'your_file_path.json' with the path to your JSON file
file_path = "reactions.json"

with open(file_path, "r") as file:
    data = json.load(file)

To constrain the output of the model, we create a simple `pydantic` class in which we define three subclasses: reactants, solvent and products. For reactants and products we also include the amount as mass or volume, and the units for the amount.

In [4]:
class Reactant(BaseModel):
    name: str
    amount: Optional[float] = Field(
        ..., description="Amount as mass or volume of the reactant"
    )
    amount_units: Optional[float]


class Product(BaseModel):
    name: str
    amount: Optional[float] = Field(
        ..., description="Amount as mass or volume of the product"
    )
    amount_units: Optional[float]


class Solvent(BaseModel):
    name: str = Field(
        "Name of the specie. If another is contained in it, definitely it is the solvent"
    )


class ReactionSpecies(BaseModel):
    reactant: List[Reactant]
    product: List[Product]
    solvent: Optional[List[Solvent]]

We also define a simple converter from IUPAC names to SMILES. This is needed since the reaction procedures from the dataset use IUPAC, traditional or commercial names to refer to the molecules while `rdkit` works preferable with SMILES.

In [5]:
CACTUS = "https://cactus.nci.nih.gov/chemical/structure/{0}/{1}"


def iupac_to_smiles(iupac):
    """
    Use the chemical name resolver https://cactus.nci.nih.gov/chemical/structure.
    If this does not work, use pubchem.
    """
    try:
        time.sleep(0.001)
        rep = "smiles"
        url = CACTUS.format(iupac, rep)
        response = requests.get(url, allow_redirects=True, timeout=10)
        response.raise_for_status()
        name = response.text
        if "html" in name:
            return None
        return name
    except Exception:
        try:
            compound = pcp.get_compounds(iupac, "name")
            return compound[0].isomeric_smiles
        except Exception:
            return None

The last function that we need is to count the number atoms of each element in the molecules.

````{margin}
The function `composition` is a modification from one presented in [`rdkit` GitHub repository discussion](https://github.com/rdkit/rdkit/discussions/5339).
````

In [6]:
def composition(molecule):
    """Get the composition of an RDKit molecule:
    Atomic counts, including hydrogen atoms, and any charge.
    For example, fluoride ion (chemical formula F-, SMILES string [F-])
    returns {9: 1, 0: -1}.

    :param molecule: The molecule to analyze
    :type some_input: An RDKit molecule
    :rtype: A dictionary.
    """
    # Add hydrogen atoms--RDKit excludes them by default
    m = Chem.MolFromSmiles(molecule)
    molecule_with_Hs = Chem.AddHs(m)
    comp = defaultdict(lambda: 0)

    # Get atom counts
    for atom in molecule_with_Hs.GetAtoms():
        comp[atom.GetAtomicNum()] += 1

    # If charged, add charge as "atomic number" 0
    charge = GetFormalCharge(molecule_with_Hs)
    if charge != 0:
        comp[0] = charge
    return comp

## Prompting and Extracting

To extract the data, we are going to prompt the model with a two-shot prompt. In addition the prompt will contain the reaction procedure to extract from.

In [7]:
system_prompt = "You are an expert in organic chemistry. Your task is to extract information about reactants and products from a given reaction procedure."

In [8]:
user_prompt = """Two examples are provided in order to help you:

Example 1:

Reaction procedure: {reaction1}
Answer: {answer1}

Example 2:

Reaction procedure: {reaction1}
Answer: {answer2}

The reaction procedure is the following:

{reaction_procedure}

Now extract the data from it according to the schema.
"""

```{margin}
The two shots used were prepared using two reaction procedures from the USPTO-ORD-100K dataset, and extracting the data by hand to formulate the answers.
```

In [9]:
reaction1 = "1-(3,4-dichlorobenzyl)-3-(4-(iodomethyl)thiazol-2-yl)urea (Intermediate 6) was taken up in tetrahydrofuran and an excess of the 2,4-dimethoxy-benzylamine (20 eq.) was added. The reaction was allowed to stir overnight at room temperature. The volatiles were removed in vacuo. Resulting oil triturated with water to give a gooey solid. Water was decanted off and resulting residue was purified by column chromatography using 0-8% gradient of 7 N ammonia/MeOH and DCM to give 1-(3,4-Dichloro-benzyl)-3-{4-[(2,4-dimethoxy-benzylamino)-methyl]-thiazol-2-yl}-urea."
answer1 = "reactant=[Reactant(name='1-(3,4-dichlorobenzyl)-3-(4-(iodomethyl)thiazol-2-yl)urea', amount=None, amount_units=None), Reactant(name='2,4-dimethoxy-benzylamine', amount=20.0, amount_units=None)] product=[Product(name='1-(3,4-Dichloro-benzyl)-3-{4-[(2,4-dimethoxy-benzylamino)-methyl]-thiazol-2-yl}-urea', amount=None, amount_units=None)] solvent=[Solvent(name='water'), Solvent(name='tetrahydrofuran')]"

reaction2 = "To a solution of (+)-trans-3-hydroxymethyl-4-phenylcyclopentan-1-one from Example 8, Step F (3.3 g, 16 mmol) in methylene chloride (100 mL) was added t-butyldimethylsilyl chloride (11 g, 49 mmol) and DIPEA (22 mL, 74 mmol). The reaction was stirred at rt for 16 h, poured into dilute aq. hydrochloric acid and extracted twice with ether. The organic layers were washed with brine, dried over sodium sulfate, a combined and concentrated. The residue was purified by FC (5% ethyl acetate in hexanes) to afford of (+)-trans-1-t-butyldimethylsilyloxymethyl-4-oxo-2-phenylcyclopentane (6.3 g) as a oil."
answer2 = "reactant=[Reactant(name='(+)-trans-3-hydroxymethyl-4-phenylcyclopentan-1-one', amount=3.3, amount_units='g'), Reactant(name='t-butyldimethylsilyl chloride', amount=11, amount_units='g'), Reactant(name='DIPEA', amount=22, amount_units='mL')] solvent=[Solvent(name='methylene chloride')] product=[Product(name='(+)-trans-1-t-butyldimethylsilyloxymethyl-4-oxo-2-phenylcyclopentane', amount=6.3, amount_units=g)]"

Finally, we do the extraction for each of the reactions considered.

In [10]:
client = instructor.patch(OpenAI(), mode=instructor.Mode.MD_JSON)

for i, reaction in enumerate(data):
    reaction_procedure = reaction["procedure_text"]
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt.format(
                reaction1=reaction1,
                answer1=answer1,
                reaction2=reaction2,
                answer2=answer2,
                reaction_procedure=reaction_procedure,
            ),
        },
    ]

    completion = client.chat.completions.create(
        model="gpt-4",
        response_model=ReactionSpecies,
        max_retries=3,
        messages=messages,
        temperature=0,
    )

    reactant_smiles = [
        iupac_to_smiles(reactant.name) for reactant in completion.reactant
    ]
    product_smiles = [iupac_to_smiles(product.name) for product in completion.product]

    # Extract the atoms number for reactants and products
    reactants_atoms = []
    for reactant in reactant_smiles:
        reactants_atoms.append(composition(reactant))

    products_atoms = []
    for product in product_smiles:
        products_atoms.append(composition(product))

    print(f"Reaction {i+1}\n")
    print(reaction_procedure)
    print(completion)
    print("\n")
    print("Atom counting:")

    # Summing up values for reactant_smiles
    sum_reactant_smiles = defaultdict(int)
    for d in reactants_atoms:
        for key, value in d.items():
            sum_reactant_smiles[key] += value

    # Summing up values for product_smiles
    sum_product_smiles = defaultdict(int)
    for d in products_atoms:
        for key, value in d.items():
            sum_product_smiles[key] += value

    # Comparing the summed values of equal keys
    for key in sum_reactant_smiles:
        if key in sum_product_smiles:
            print(
                f"{periodictable.elements[key]}: Reactant = {sum_reactant_smiles[key]}, Product = {sum_product_smiles[key]}"
            )
        else:
            print(f"{periodictable.elements[key]} not present in Products")
    print("\n\n")

Reaction 1

A solution of 0.55 g (1.6 mmol) (S)-7-amino-5-(4-methoxy-benzyl)-5H,7H-dibenzo[b,d]azepin-6-one, 3.74 ml (50 mmol) trifluoroacetic acid and 1.4 ml (16 mmol) trifluormethane sulfonic acid in 38 ml dichloromethane was stirred at room temperature for 4 hours. The solvent was distilled off and extraction with aqueous sodium bicarbonate solution/ethyl acetate followed by chromatography on silicagel with ethylacetate/methanol (100-95/0-5) yielded 0.35 g (96%) (S)-7-amino-5H,7H-dibenzo[b,d]azepin-6-one as orange solid; MS: m/e: 225.4 (M+H+).
reactant=[Reactant(name='(S)-7-amino-5-(4-methoxy-benzyl)-5H,7H-dibenzo[b,d]azepin-6-one', amount=0.55, amount_units=None), Reactant(name='trifluoroacetic acid', amount=3.74, amount_units=None), Reactant(name='trifluormethane sulfonic acid', amount=1.4, amount_units=None)] product=[Product(name='(S)-7-amino-5H,7H-dibenzo[b,d]azepin-6-one', amount=0.35, amount_units=None)] solvent=[Solvent(name='dichloromethane'), Solvent(name='aqueous sodium b

If we manually analyze the results, we can see that the extraction is mainly good for the species names.  

The model correctly identify all the reactants and products for the three reactions. Nevertheless, it fails identifying the solvents by including as solvent some species used for separation and purification of the products. 

In addition, the model has some problems to correctly identify the units of the amount, specially when they are as volume. 

However, despite the extraction is correct, the atom count is not fulfilled for any of the reactions. This is because the reaction procedure omit all the product that are not the final desired product. To solve this, one can incorporate this to an agent environment, in which the agent then can retrieve all the species from a reaction database for example.

## Bibliography

```{bibliography}
:filter: docname in docnames
```